# EEG - Flow

## 7. Epochs Evokeds


Last edit: 17.04.2023 19:37
@anguyen

In [ ]:
from datetime import datetime

import numpy as np
from matplotlib import pyplot as plt # viz

from mne import read_evokeds
from mne.viz import set_browser_backend

from eeg_flow.config import load_config
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files

%matplotlib qt
set_browser_backend('qt')

_, derivatives_folder_root, experimenter = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
PARTICIPANT = 19        # int
GROUP       = 6         # int [1, 2, 3, 4, 5, 6, 7, 8]
TASK        = "oddball" # str [oddball, UT]
RUN         = 2         # int [1, 2]

###############################################################################

DERIVATIVES_FOLDER = get_folder(DERIVATIVES_FOLDER_ROOT, PARTICIPANT, GROUP)
FNAME_STEM = get_fname(PARTICIPANT, GROUP, TASK, RUN)
DERIVATIVES_SUBFOLDER = DERIVATIVES_FOLDER / FNAME_STEM

# create locks
derivatives = []

locks = lock_files(*derivatives)

# load previous steps
fname_ev_standard = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step6_e1-standard_evoked-ave.fif")
fname_ev_target   = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step6_e2-target_evoked-ave.fif")
fname_ev_novel    = DERIVATIVES_SUBFOLDER / (FNAME_STEM + "_step6_e3-novel_evoked-ave.fif")
all_evokeds = dict()

all_evokeds["standard"] = read_evokeds(fname_ev_standard,  condition=0)
all_evokeds["target"]   = read_evokeds(fname_ev_target,    condition=0)
all_evokeds["novel"]    = read_evokeds(fname_ev_novel,     condition=0)


## 7.1 

In [ ]:
event_id = dict(standard=1, target=2, novel=3)

In [ ]:
max_peak = False
min_peak = False

for event in event_id:
    all_evokeds[event].drop_channels(all_evokeds[event].info['bads']) #remove bads from this report)
    event_max = np.amax(all_evokeds[event].data)
    event_min = np.amin(all_evokeds[event].data)

    if max_peak == False or max_peak < event_max:
        max_peak = event_max
    if min_peak == False or min_peak > event_min:
        min_peak = event_min

min_peak *=10**6
max_peak *=10**6

print(min_peak, max_peak)

ylim_values = {}

for event in event_id:
    event_max = np.amax(all_evokeds[event].data)
    event_min = np.amin(all_evokeds[event].data)
    ylim_values[event] = [event_max * 1.2 * 10**6, event_min * 1.2 * 10**6]
ylim_values


In [ ]:
## use this if limits need to be set manually
max_peak, min_peak = 13, -10

f, ax = plt.subplots(3, 1, figsize=(8, 8))
f.suptitle(
    FNAME_STEM
    + " | All evoked | All channels | Fixed scale | ("
    + str(min_peak)
    + ", "
    + str(max_peak)
    + ")"
)

ylim = dict(eeg=[max_peak * 1.2, min_peak * 1.2])

for k, (condition, evo) in enumerate(all_evokeds.items()):
    print(k, condition, evo)
    evo.plot(axes=ax[k], ylim=ylim, zorder="std")
    ax[k].set_title(condition.capitalize())
f.tight_layout()

timestampStr = datetime.now().strftime("%Y_%m_%d_%H_%M")
FNAME_BRIDGE_PLOT = (
    DERIVATIVES_SUBFOLDER
    / "plots"
    / (FNAME_STEM + "_step7_allElec_fixedScale-" + timestampStr + ".svg")
)
f.savefig(FNAME_BRIDGE_PLOT)

# fig_all_evoked_fixed = f
# fig_all_evoked_fixed


In [ ]:
# ## this is for a zoomed in view for each plot
# ##this would have been nice but we need the y axis to be swapped.
# ## until there's a clean way to do it, use the next cell instead

# f, ax = plt.subplots(3, 1, figsize=(8, 8))
# f.suptitle(FNAME_STEM + " | All evoked | All channels | Adaptive scale | (" + str(min_peak) +", "+str(max_peak)+")")

# for k, (condition, evo) in enumerate(all_evokeds.items()):
#     print(k, condition, evo)
#     evo.plot(axes=ax[k])
#     ax[k].set_title(condition.capitalize())
# f.tight_layout()

# timestampStr = datetime.now().strftime("%Y_%m_%d_%H_%M")
# # fname_bridge_plot = DERIVATIVES_SUBFOLDER / "plots" / (FNAME_STEM + "_step7_allElec_AdaptiveScale-"+timestampStr+".svg")
# # f.savefig(fname_bridge_plot)

# fig_all_evoked_fixed = f
# fig_all_evoked_fixed

In [ ]:
fig = plt.figure(figsize=(8, 8))

ax1 = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)

fig.suptitle(FNAME_STEM + " | All evoked | All channels | Adaptive scale")

ylim_standard = dict(eeg=ylim_values["standard"])
ylim_target = dict(eeg=ylim_values["target"])
ylim_novel = dict(eeg=ylim_values["novel"])

all_evokeds["standard"].plot(
    picks='eeg',
    axes=ax1,
    spatial_colors=True,
    zorder="std",
    ylim = ylim_standard,
    titles="standard"
)
all_evokeds["target"].plot(
    picks='eeg',
    axes=ax2,
    spatial_colors=True,
    zorder="std",
    ylim = ylim_target,
    titles="target"
)
all_evokeds["novel"].plot(
    picks='eeg',
    axes=ax3, 
    spatial_colors=True,
    zorder="std",
    ylim = ylim_novel,
    titles="novel")
fig.tight_layout()

timestampStr = datetime.now().strftime("%Y_%m_%d_%H_%M")
FNAME_BRIDGE_PLOT2 = DERIVATIVES_SUBFOLDER / "plots" / (FNAME_STEM + "_step7_allElec_AdaptiveScale-"+timestampStr+".svg")
fig.savefig(FNAME_BRIDGE_PLOT2)
fig

## 7.2 Epochs


## 7.3 Save the rest of the derivatives

The ICA decomposition can be saved.

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway